In [1]:
import pandas as pd

from src import akeneo, config

In [2]:
data_dir = config.dir_data / "dataset"

In [3]:
client = akeneo.create_client_from_env()

## Categories

In [4]:
categories_df = pd.read_csv(data_dir / "categories.csv")
categories_df

,parent,code,en,de
0,NaN,master,Master,Master
1,master,samsung,Samsung,Samsung
2,samsung,s20_series,S20 Series,S20-Reihe
3,s20_series,s20,S20,S20
4,s20_series,s20_plus,S20+,S20+
5,s20_series,s20_ultra,S20 Ultra,S20 Ultra
6,s20_series,s20_fe,S20 FE,S20 FE
7,samsung,s21_series,S21 Series,S21-Reihe
8,s21_series,s21,S21,S21
9,s21_series,s21_plus,S21+,S21+


In [5]:
for _, parent, code, en, de in categories_df.itertuples():
    res = client.request(
        "pim_api_category_partial_update",
        {"code": code},
        {
            "parent": parent if pd.notna(parent) else None,
            "labels": {
                "en_US": en,
                "en_GB": en,
                "de_DE": de,
            },
        },
    )
    print(code, "=>", res)

master => {'status': 204, 'message': 'No Content'}
samsung => {'status': 204, 'message': 'No Content'}
s20_series => {'status': 204, 'message': 'No Content'}
s20 => {'status': 204, 'message': 'No Content'}
s20_plus => {'status': 204, 'message': 'No Content'}
s20_ultra => {'status': 204, 'message': 'No Content'}
s20_fe => {'status': 204, 'message': 'No Content'}
s21_series => {'status': 204, 'message': 'No Content'}
s21 => {'status': 204, 'message': 'No Content'}
s21_plus => {'status': 204, 'message': 'No Content'}
s21_ultra => {'status': 204, 'message': 'No Content'}
s21_fe => {'status': 204, 'message': 'No Content'}
s22_series => {'status': 204, 'message': 'No Content'}
s22 => {'status': 204, 'message': 'No Content'}
s22_plus => {'status': 204, 'message': 'No Content'}
s22_ultra => {'status': 204, 'message': 'No Content'}


## Products

In [6]:
products_file_df = pd.read_csv(data_dir / "products.csv")
products_file_df

,family,brand,categories,url
0,mobile_phone_cases,Mobiparts,s20,https://icecat.biz/en/p/mobiparts/104889/mobil...
1,mobile_phone_cases,Mobiparts,s20,https://icecat.biz/en/p/mobiparts/mp-104927/mo...
2,mobile_phone_cases,Mobiparts,s20,https://icecat.biz/en/p/mobiparts/mp-104925/mo...
3,mobile_phone_cases,Mobiparts,s20,https://icecat.biz/en/p/mobiparts/mp-108422/mo...
4,mobile_phone_cases,Mobiparts,s20,https://icecat.biz/en/p/mobiparts/mp-104926/mo...
...,...,...,...,...
75,mobile_phone_cases,PanzerGlass,s20_ultra,https://icecat.biz/en/p/panzerglass/0237/mobil...
76,mobile_phone_cases,PanzerGlass,s22,https://icecat.biz/en/p/panzerglass/0375/mobil...
77,mobile_phone_cases,PanzerGlass,s22_ultra,https://icecat.biz/en/p/panzerglass/0376/mobil...
78,mobile_phone_cases,PanzerGlass,s22,https://icecat.biz/en/p/panzerglass/0371/mobil...


In [7]:
products_file_df.describe()

,family,brand,categories,url
count,80,80,80,80
unique,1,4,11,80
top,mobile_phone_cases,Samsung,s20,https://icecat.biz/en/p/mobiparts/104889/mobil...
freq,80,33,18,1


In [8]:
def get_sku_and_ean(url: str):
    return (
        url,
        url.split("/")[6].upper(),
        url.split("/")[7].split("-")[1],
    )

sku_and_ean_df = pd.DataFrame.from_records(products_file_df["url"].map(get_sku_and_ean), columns=["url", "sku", "ean"])

products_df = pd.merge(
    products_file_df,
    sku_and_ean_df,
    on="url"
)
products_df["categories"] = products_df["categories"].str.split(",")
products_df

,family,brand,categories,url,sku,ean
0,mobile_phone_cases,Mobiparts,[s20],https://icecat.biz/en/p/mobiparts/104889/mobil...,104889,8718066381026
1,mobile_phone_cases,Mobiparts,[s20],https://icecat.biz/en/p/mobiparts/mp-104927/mo...,MP-104927,8718066381170
2,mobile_phone_cases,Mobiparts,[s20],https://icecat.biz/en/p/mobiparts/mp-104925/mo...,MP-104925,8718066381156
3,mobile_phone_cases,Mobiparts,[s20],https://icecat.biz/en/p/mobiparts/mp-108422/mo...,MP-108422,8718066383501
4,mobile_phone_cases,Mobiparts,[s20],https://icecat.biz/en/p/mobiparts/mp-104926/mo...,MP-104926,8718066381163
...,...,...,...,...,...,...
75,mobile_phone_cases,PanzerGlass,[s20_ultra],https://icecat.biz/en/p/panzerglass/0237/mobil...,0237,5711724002373
76,mobile_phone_cases,PanzerGlass,[s22],https://icecat.biz/en/p/panzerglass/0375/mobil...,0375,5711724003752
77,mobile_phone_cases,PanzerGlass,[s22_ultra],https://icecat.biz/en/p/panzerglass/0376/mobil...,0376,5711724003769
78,mobile_phone_cases,PanzerGlass,[s22],https://icecat.biz/en/p/panzerglass/0371/mobil...,0371,5711724003714


In [9]:
for _, family, brand, categories, url, sku, ean in products_df.itertuples():
    res = client.request(
        "pim_api_product_partial_update",
        {"code": sku},
        {
            "family": family,
            "categories": categories,
            "values": {"ean": [{"locale": None, "scope": None, "data": ean}]},
        },
    )
    print(sku, "=>", res)

104889 => {'status': 204, 'message': 'No Content'}
MP-104927 => {'status': 204, 'message': 'No Content'}
MP-104925 => {'status': 204, 'message': 'No Content'}
MP-108422 => {'status': 204, 'message': 'No Content'}
MP-104926 => {'status': 204, 'message': 'No Content'}
MP-104938 => {'status': 204, 'message': 'No Content'}
MP-104924 => {'status': 204, 'message': 'No Content'}
MP-107808 => {'status': 204, 'message': 'No Content'}
MP-108423 => {'status': 204, 'message': 'No Content'}
MP-107805 => {'status': 204, 'message': 'No Content'}
MP-104939 => {'status': 204, 'message': 'No Content'}
MP-107806 => {'status': 204, 'message': 'No Content'}
MP-110885 => {'status': 204, 'message': 'No Content'}
MP-112908 => {'status': 204, 'message': 'No Content'}
MP-112914 => {'status': 204, 'message': 'No Content'}
MP-112920 => {'status': 204, 'message': 'No Content'}
MP-112944 => {'status': 204, 'message': 'No Content'}
MP-112940 => {'status': 204, 'message': 'No Content'}
MP-112939 => {'status': 204, 'm

In [11]:
products_in_pim_df = pd.DataFrame(client.request("pim_api_product_list"))
products_in_pim_df

,identifier,enabled,family,categories,groups,parent,values,created,updated,associations,quantified_associations
0,MP-104925,True,mobile_phone_cases,[s20],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:14:51+00:00,2022-02-24T13:14:51+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
1,MP-108422,True,mobile_phone_cases,[s20],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:14:51+00:00,2022-02-24T13:14:51+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
2,MP-104926,True,mobile_phone_cases,[s20],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:14:51+00:00,2022-02-24T13:14:51+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
3,MP-104938,True,mobile_phone_cases,[s20],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:14:51+00:00,2022-02-24T13:14:51+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
4,MP-104924,True,mobile_phone_cases,[s20],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:14:52+00:00,2022-02-24T13:14:52+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
...,...,...,...,...,...,...,...,...,...,...,...
74,0376,True,mobile_phone_cases,[s22_ultra],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:15:10+00:00,2022-02-24T13:15:10+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
75,0371,True,mobile_phone_cases,[s22],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:15:10+00:00,2022-02-24T13:15:10+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
76,0372,True,mobile_phone_cases,[s22_plus],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:15:10+00:00,2022-02-24T13:15:10+00:00,"{'PACK': {'products': [], 'product_models': []...",{}
77,EF-RG998CBEGWW,True,mobile_phone_cases,[s21_ultra],[],None,"{'ean': [{'locale': None, 'scope': None, 'data...",2022-02-24T13:38:51+00:00,2022-02-24T13:38:51+00:00,"{'PACK': {'products': [], 'product_models': []...",{}


In [21]:
products_df[
    ~products_df["sku"].isin(products_in_pim_df["identifier"])
]

,family,brand,categories,url,sku,ean
45,mobile_phone_cases,Samsung,[s20_plus],https://icecat.biz/en/p/samsung/ef-ng985pjegeu...,EF-NG985PJEGEU,8806090292262


In [22]:
client.request(
    "pim_api_product_partial_update",
    {"code": "EF-NG985PJEGEU"},
    {
        "family": "mobile_phone_cases",
        "categories": ["s20_plus"],
        "values": {"ean": [{"locale": None, "scope": None, "data": "8806090292262"}]},
    },
)

{'status': 201, 'message': 'Created'}